In [5]:
from chembl_webresource_client.new_client import new_client
import pandas as pd
import numpy as np

#mTOR
target_id = "CHEMBL2842"

#check activities with IC50 values
activity = new_client.activity.filter(
    target_chembl_id=target_id
).filter(
    standard_type="IC50"
)

df = pd.DataFrame(activity)
print("raw entries found:", len(df))
df.head()

raw entries found: 5269


,action_type,activity_comment,activity_id,activity_properties,assay_chembl_id,assay_description,assay_type,assay_variant_accession,assay_variant_mutation,bao_endpoint,...,target_organism,target_pref_name,target_tax_id,text_value,toid,type,units,uo_units,upper_value,value
0,None,None,866953,[],CHEMBL682171,The inhibitory activity by using FK506 binding...,B,None,None,BAO_0000190,...,Homo sapiens,Serine/threonine-protein kinase mTOR,9606,None,None,Log IC50,None,UO_0000065,None,-5.0
1,None,None,868131,[],CHEMBL682171,The inhibitory activity by using FK506 binding...,B,None,None,BAO_0000190,...,Homo sapiens,Serine/threonine-protein kinase mTOR,9606,None,None,Log IC50,None,UO_0000065,None,-5.0
2,None,None,873239,[],CHEMBL682171,The inhibitory activity by using FK506 binding...,B,None,None,BAO_0000190,...,Homo sapiens,Serine/threonine-protein kinase mTOR,9606,None,None,Log IC50,None,UO_0000065,None,-5.0
3,None,None,873240,[],CHEMBL677281,The inhibitory activity by using FK506 binding...,B,None,None,BAO_0000190,...,Homo sapiens,Serine/threonine-protein kinase mTOR,9606,None,None,Log IC50,None,UO_0000065,None,-8.5
4,None,None,875431,[],CHEMBL682171,The inhibitory activity by using FK506 binding...,B,None,None,BAO_0000190,...,Homo sapiens,Serine/threonine-protein kinase mTOR,9606,None,None,Log IC50,None,UO_0000065,None,-5.0


In [7]:
#drop rows w/o values or SMILES
df = df[df["standard_value"].notna()]
df = df[df["canonical_smiles"].notna()]
df = df[df["standard_units"] == "nM"]

#remove outliers (IC50 > 1,000,000 nM)
df = df[df["standard_value"].astype(float) < 1_000_000]

#IC50 nM --> pIC50
df["standard_value"] = df["standard_value"].astype(float)
df["pIC50"] = -df["standard_value"].apply(lambda x: np.log10(x * 1e-9))

df_final = df[["canonical_smiles", "pIC50"]].dropna()
print("cleaned dataset size:", df_final.shape)
df_final.head()

cleaned dataset size: (5169, 2)


,canonical_smiles,pIC50
0,CC(CN1CCCCC1)OC(=O)[C@@H]1CCCN1C(=O)C(=O)C(C)(C)C,5.000000
1,CC(CN1CCCCC1)OC(=O)[C@@H]1CCCN1C(=O)C(=O)c1ccccc1,5.000000
2,CC(C)CC(=O)C(=O)N1CCC[C@H]1C(=O)OCCCc1cccnc1,5.000000
3,C=CC[C@@H]1/C=C(\C)C[C@H](C)C[C@H](OC)[C@H]2O[...,8.500038
4,CC(C)CC(=O)C(=O)N1CCC[C@H]1C(=O)OCCS(=O)(=O)c1...,5.000000


In [8]:
df_final.to_csv("../data/bioactml_data.csv", index=False)
print("Saved to ../data/bioactml_data.csv")

Saved to ../data/bioactml_data.csv
